## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
# bi - bank_info
bi = pd.read_csv('/datasets/data.csv')
bi.info()
bi.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Мы прочитали файл, посмотрели общую информацию и вывели первые 15 строк. 
Сразу обращаем внимание на:
  
  -- 1) Значения NaN в столбцах *days_employed* и *total_income*, обозначают судя по всему безработных
  
  -- 2) У этих же столбцов не удобный для этого случая тип данных float64 - лучше заменить его на int64
  
  3) Некорректные или отрицательные значения в столбце *days_employed*
  
  -- 4) в столбце *education* видим разницу регистров, для выявления дубликатов стоит все значения привести к одному
  
  -- 5) в столбце *purpose* схожие цели описаны по-разному, стоит провести лемматизацию и разделить их по группам 
  
  6) столбцы *children* и *dob_years* стоит проверить на максимальное и минимальное значение, во избежание ошибок

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
bi.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
bi = bi.fillna(0)
bi.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
purpose_topic       0
children_exs        0
gold                0
dtype: int64

### Вывод

Пропущенные значения относились к безработным клиентам.
          
   Пропуски могли появиться из-за некорретно распознаных данных, значение 0 могли принять за пропуск (NaN)
                      
   Пропуски заполняем по принципу безработный клиент - это клиент со значением *Стаж* и *ежемесячный доход* = 0


### Замена типа данных

In [5]:
bi['days_employed'] = bi['days_employed'].astype('int')
bi['total_income'] = bi['total_income'].astype('int')
bi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Заменяем тип данных на более удобный в этом случае int64, с помощью метода *astype*, так как первод нужно сделать из уже числового метода float64

### Обработка дубликатов

In [6]:
bi.duplicated().sum()
#изначально дубликатов только 54, но пороверим их количество после приведения строк стобца 'education' к нижнему регистру

54

In [7]:
bi['education'] = bi['education'].str.lower()
bi.duplicated().sum()

71

In [8]:
#71! теперь можем спокойно удалять все дубликаты, считать одного чеовека дважды, нам точно не нужно)
bi.drop_duplicates(inplace=True)
bi.duplicated().sum()

0

### Вывод

Для выяснения дубликатов используем сначала метод *.str.lower()* для приведения к единому регистру всех значений в столбце, затем методы *.duplicated()* и *.sum()* для выявления и подчета дубликатов
в конце удаляем их методом *.drop_duplicates* с аргументом *(inplace=True)*

Дубликаты в данных могут появлять из-за некорректно набранных данных или из-за не удачно соединённых таблиц

### Лемматизация

Лемматизируем значения столбца *purpose* и создадим новый столбец *purpose_topic* с общими темами взятия кредита -

  1) Недвижимость
  2) Автомобиль
  3) Свадьба
  4) Образование

In [9]:
#сначала импортируем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()
lemmas = []

In [10]:
#создадим список уникальных целей кредита purpose_uniq
purpose_uniq = bi['purpose'].unique()
print(purpose_uniq)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [11]:
#лемматизируем purpose_uniq в новом списке purpose_lemm
purpose_lemm = []
for i in purpose_uniq:
    lemma = m.lemmatize(i)
    purpose_lemm.append(lemma)
print(purpose_lemm)

[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['жилье', '\n'], ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'], ['автомобиль', '\n'], ['заниматьс

### Вывод

Мы лемматизировали все значения столбца "цели кредита" в отдельный столбец, теперь мы можем категоризировать данные и пртсоединить их в новом столбце *purpose_topic*

### Категоризация данных

In [12]:
#создаём функцию выявления лемм, по четырем категориям - "недвижимость"(или жильё), "автомобиль", "Свадьба" и "образование"
bi['purpose_topic'] = ''
def lemm(row):
    i = m.lemmatize(row)
    if 'автомобиль' in i:
        return 'автомобиль'
    if 'свадьба' in i:
        return  'свадьба'
    if 'образование' in i:
        return 'образование'
    return 'недвижимость'
bi['purpose_topic']= bi['purpose'].apply(lemm)        
bi.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_topic
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Вывод

Данные успешно категоризированны! 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [13]:
#мы видели что есть клиенты со значением детей -1, давайте их подсчитаем
print('клиенты с -1 ребенком:', bi[bi['children'] == -1].count()[0])

клиенты с -1 ребенком: 47


In [14]:
#47 не критично много, но чтобы не портить статистику заменим их на значение "1"
bi['children'] = bi['children'].replace(-1, 0)

In [15]:
#посмотрим сколько клиентов со значением 20
print("Количество строк с 'children' 20:", bi[bi['children'] == 20].count()[0])
#76 тоже не критично, будем считать что они просто многодетные

Количество строк с 'children' 20: 76


Теперь определим влияние количества детей на кредитнуюспособность 
нужно определить 2 категории "с детьми" и "без детей" узнать в какой категории больше значений %*debt*>0 ,и сделать вывод

In [16]:
#добавим столбец с категориями
def children_exs(row):
    if row['children'] == 0:
        return 'нет детей'
    else:
        return 'есть дети'
bi['children_exs'] = bi.apply(children_exs, axis=1)
#теперь напишем функцию выясняющую среднее значение долгов по категории *children*
def result(row):
    return bi.groupby(row)['debt'].mean().to_frame().sort_values(by='debt')                                                                             
result('children_exs')



,debt
children_exs,
нет детей,0.075258
есть дети,0.092537


### Вывод

Таким образом мы видим, что вреднем клиенты *с детьми* чаще имеют задолжности - а значит мнеее привлекательны для банка!

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
result('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


### Вывод

Клиенты не состоящие в *браке* - чаще имеют задолжности - а значит мнеее привлекательны для банка!
В отличии от клиентов которые остались *вдовцом / вдовой*, они имеют самый низкий шанс задержки по оплате кредита
Женатые, замужние и разведенные клиенты тоже городо чаще платят креды в срок!



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

In [18]:
#Сначала распредилим клиентов по 3 категориям - нижний, средний и высокий
def need_more_gold(row):
    if row['total_income'] <= 50000:
        return 'нижний'
    elif 50000 < row['total_income'] <= 150000:
        return 'средний'
    else:
        return 'высокий'

bi['gold'] = bi.apply(need_more_gold, axis=1)
bi.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_topic,children_exs,gold
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,есть дети,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,средний
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,средний
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,высокий
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,нет детей,высокий
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,нет детей,высокий
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей,средний
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,есть дети,средний
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей,средний


In [21]:
result('gold')

,debt
gold,
нижний,0.077980
высокий,0.079051
средний,0.083920


### Вывод

Клиенты со *средним* доходом - чаще имеют задолжности, в вот клиенты с *низким* уровнем дохода наоборот, реже не отдают долг банку!

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
result('purpose_topic')

,debt
purpose_topic,
недвижимость,0.072334
свадьба,0.080034
образование,0.092200
автомобиль,0.093590


### Вывод

Клиенты берущие кредиты на операции с *недвижимостью* самые надежные! 
А вот те кто берет деньги для покупки *автомобиля* или платы за *образование*, наоборот становятся должниками - чаще

### Шаг 4. Общий вывод

Проведя анализ, можно сказать, что в среднем лучшим кандидатом для кредита станет:
Овдовевший, бездетный клиент с низким доходом, берущий кредит на недвижимость.

Наоборот менее выгодно будет давать кредит:
Не женатому / не замужнему (или состоящему в гражданском браке), киленту с ребенком, со средним доходом - на автомобиль или образование

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.